In [1]:
import itertools
import numpy as np
import pandas as pd
import sys
import os
from datetime import timedelta

print(os.getcwd())

C:\Users\apram\hodschallenge


In [2]:
!python -m pip install pandas

In [3]:
in_file = "challenge/data/device_activations.csv"

In [4]:
previous_readings = pd.read_csv(in_file)
print(type(previous_readings['time'][0]))
previous_readings['time'] = pd.to_datetime(previous_readings['time'])
device_list = sorted(previous_readings['device'].unique())
n_devices = len(device_list)
print(type(previous_readings['time'][0]))
print(previous_readings.loc[0, :]['time'])
print(previous_readings.loc[1, :]['time'])
print(previous_readings.loc[2, :]['time'])

<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2016-07-01 04:23:32
2016-07-01 06:52:57
2016-07-01 06:53:00


In [5]:
print("Number of data points: ", len(previous_readings))

Number of data points:  9045


In [6]:
def date_time_same_hour(dt1, dt2):
    same_until_hour = dt1.year == dt2.year and dt1.month == dt2.month and dt1.day == dt2.day and dt1.hour == dt2.hour
    if same_until_hour:
        return True
    else:
        return False


In [7]:
# IDEA: ADD HOUR & DAY AS FEATURE?
# 541 hours with positive label
# Weekday as 7 binary inputs?
def preallocate_features(previous_readings):
    n_devices = previous_readings['device'].nunique()
    print('n_devices = ', n_devices)
    tmp_time_stamp = first_time_stamp = pd.to_datetime(previous_readings['time'][0])
    
    unique_hour_date_times = [first_time_stamp.replace(minute=0, second=0)]
    
    hour_interval_start_end = pd.date_range(first_time_stamp.replace(minute=0, second=0), 
                                            previous_readings['time'][len(previous_readings) - 1].replace(minute=0, second=0), 
                                           freq='H')
    
    features = pd.DataFrame(0, index=np.arange(len(hour_interval_start_end)), 
                            columns=['time', 'weekday', 'hour'] + device_list)
    features['time'] = hour_interval_start_end
    return features
#print("Unique hours: ", len(preallocate_features(previous_readings)))
#print(preallocate_features(previous_readings))
    

In [8]:
def preprocess_features_and_labels(previous_readings):
    """
    Generate features so that we have an array with dimensions [T, n_devices, 1]
    T: time
    n_devices: number of devices
    Last dimension: number of activations per hour
    """
    
    # We preallocate to avoid many appends (append copies according to pandas docs, might become an issue/slow for large data)
    features = preallocate_features(previous_readings)
    labels = pd.DataFrame(0, index=np.arange(len(features) - 1), 
                            columns=device_list)
    for index, row in previous_readings.iterrows():
        if index == 0:
            continue
            
        dt = row['time'].replace(minute=0, second=0)
        feature_idx = features.index[features['time'] == dt]
        # Increment device's counter at time
        features.loc[feature_idx, row['device']] += 1
        labels.loc[feature_idx - 1, row['device']] = 1
            
        features['time']
    
    #Second loop, can we improve here?
    for index, row in features.iterrows():
        features.loc[index, 'weekday'] = row['time'].weekday()
        features.loc[index, 'hour'] = row['time'].hour
        
        
    features.drop('time', axis=1, inplace=True)
    
    return (features, labels)

features, labels = preprocess_features_and_labels(previous_readings)

n_devices =  7


In [9]:
#print("Features")
#print(features)

#print("Labels")
#print(labels)

In [10]:
def create_timeseries_batches(features, labels, length=31):
    np_features = features.to_numpy()[:-1]
    np_labels = labels.to_numpy()
    
    print(np_features.shape)
    print(np_labels.shape)
    
    np_features = np_features[:length*(len(np_features)//length)]
    np_labels = np_labels[:length*(len(np_labels)//length)]
    
    print(np_features.shape)
    print(np_labels.shape)
    
    feature_batch = np.reshape(np_features, [-1, length, np_features.shape[1]])
    label_batch = np.reshape(np_labels, [-1, length, np_labels.shape[1]])
    
    print(feature_batch.shape)
    print(label_batch.shape)
    
    return feature_batch, label_batch

feature_batch, label_batch = create_timeseries_batches(features, labels)

train_ratio = 0.7
train_len = int(train_ratio * len(feature_batch))
print("Train len: ", train_len)

train_feature_batch = feature_batch[:train_len]
test_feature_batch = feature_batch[train_len:]
train_label_batch = label_batch[:train_len]
test_label_batch = label_batch[train_len:]

(1477, 9)
(1477, 7)
(1457, 9)
(1457, 7)
(47, 31, 9)
(47, 31, 7)
Train len:  32


In [11]:
def calc_ratio_positive_outputs_per_device(labels):
    np_labels = labels.to_numpy()[:-1]
    print(np_labels.shape)
    ratio_per_device = np.sum(np_labels, axis=0) / np_labels.shape[0]
    print("Ratio per device: ", ratio_per_device)
    return np.array(ratio_per_device)
ratio_positive_outputs_per_device = calc_ratio_positive_outputs_per_device(labels)

(1476, 7)
Ratio per device:  [0.14769648 0.26287263 0.15785908 0.18631436 0.20121951 0.3299458
 0.04471545]


In [12]:
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras import objectives
from keras import backend as K

from sklearn.metrics import mean_squared_error
from math import sqrt

from bayes_opt import BayesianOptimization

Using TensorFlow backend.


In [13]:
USE_WEIGHTED_LOSS = True

In [14]:
# Create our own weighted loss to combat label imbalance
def weighted_loss(y_true, y_pred):
    out = -(y_true * K.log(y_pred + 1e-5) / ratio_positive_outputs_per_device + (1.0 - y_true) * K.log(1.0 - y_pred + 1e-5))
    return K.mean(out, axis=-1)

In [21]:
BATCH_SIZE=1
def create_model(params):
    model = Sequential()
    model.add(LSTM(32, batch_input_shape=(params['batch_size'], None, n_devices + 2), return_sequences=True, stateful=True))
    model.add(Dropout(params['dropout']))
    model.add(Dense(n_devices, activation='sigmoid'))
    model.compile(loss=weighted_loss if params['use_weighted_loss'] else 'binary_crossentropy', optimizer=params['optimizer'])
    return model

training_params = {'optimizer': 'adam', 
                   'use_weighted_loss': USE_WEIGHTED_LOSS,
                   'batch_size': BATCH_SIZE,
                   'dropout': 0.0,
                   'epochs': 50}

#model = create_model(training_params)
#model.fit(train_feature_batch, train_label_batch, epochs=training_params['epochs'], batch_size=training_params['batch_size'], verbose=1, shuffle=False)

In [22]:
test_feature_batch_flattened = test_feature_batch.reshape([-1, *test_feature_batch.shape[-1:]])
test_label_batch_flattened = test_label_batch.reshape([-1, *test_label_batch.shape[-1:]])
test_feature_batch_expanded = test_feature_batch_flattened if len(test_feature_batch_flattened.shape) == 3 else np.expand_dims(test_feature_batch_flattened, axis=1)

In [23]:
def eval_model_params(params, train_X, train_Y, test_X, test_Y):
    model = create_model(training_params)
    history = model.fit(train_X, train_Y, validation_data=(test_X, np.expand_dims(test_Y, 1)), epochs=params['epochs'], batch_size=params['batch_size'], verbose=1, shuffle=False)
    predictions = model.predict(test_X, batch_size=1)
    return model, history.history['val_loss'][-1]

model, result = eval_model_params(training_params, train_feature_batch, train_label_batch, test_feature_batch_expanded,test_label_batch_flattened)
print(result)

Train on 32 samples, validate on 465 samples
Epoch 1/50
32/32 [==============================] - 1s 42ms/step - loss: 1.2112 - val_loss: 1.0869
Epoch 2/50
32/32 [==============================] - 1s 16ms/step - loss: 1.0265 - val_loss: 0.9607
Epoch 3/50
32/32 [==============================] - 0s 15ms/step - loss: 0.8952 - val_loss: 0.8378
Epoch 4/50
32/32 [==============================] - 0s 15ms/step - loss: 0.7840 - val_loss: 0.7424
Epoch 5/50
32/32 [==============================] - 0s 15ms/step - loss: 0.6987 - val_loss: 0.6769
Epoch 6/50
32/32 [==============================] - 0s 15ms/step - loss: 0.6355 - val_loss: 0.6254
Epoch 7/50
32/32 [==============================] - 0s 15ms/step - loss: 0.5867 - val_loss: 0.5861
Epoch 8/50
32/32 [==============================] - 0s 15ms/step - loss: 0.5495 - val_loss: 0.5599
Epoch 9/50
32/32 [==============================] - 0s 15ms/step - loss: 0.5226 - val_loss: 0.5409
Epoch 10/50
32/32 [==============================] - 0s 15ms/ste

In [24]:
print(test_feature_batch_expanded.shape)
predictions = model.predict(test_feature_batch_expanded, batch_size=1)[:, 0, :]
#print(np.round(predictions, 1))

print(predictions.shape)
print(test_label_batch_flattened.shape)


(465, 1, 9)
(465, 7)
(465, 7)


In [25]:
print("Training accuracy: ", np.sum(np.round(predictions) == test_label_batch_flattened) / predictions.size) 
print("Training accuracy per device:: ", np.sum(np.round(predictions) == test_label_batch_flattened, axis=0) / predictions.shape[0]) 

print("% of 1 prediction outputs", np.sum(np.round(predictions)) / predictions.size) 
print("% of 1 label outputs", np.sum(np.round(test_label_batch_flattened)) / test_label_batch_flattened.size)

Training accuracy:  0.8740399385560675
Training accuracy per device::  [0.90967742 0.92688172 0.84516129 0.80860215 0.85591398 0.92043011
 0.8516129 ]
% of 1 prediction outputs 0.25099846390168973
% of 1 label outputs 0.17357910906298002


In [26]:
print(test_feature_batch_expanded.shape)
print(np.expand_dims(test_label_batch_flattened, 0).shape)
model.evaluate(np.expand_dims(test_feature_batch_flattened, 1), np.expand_dims(test_label_batch_flattened, 1))

(465, 1, 9)
(1, 465, 7)


ValueError: In a stateful network, you should only pass inputs with a number of samples that can be divided by the batch size. Found: 465 samples

In [27]:
model.save('model.h5')